<a href="https://colab.research.google.com/github/AHHHZ975/banmo/blob/main/BANMo_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Using this notebook, I try to run several experiments on BANMo code to understand better their work**

# **Restart kernel**

In [ ]:
!rm -rf /content/banmo/
import os
os._exit(00)
#!pip list -v

# **0- Requirement installation**

This cell downloads the BANMo code and install required packages

In [ ]:
# Clone the BANMo repository
import os
!git clone https://github.com/facebookresearch/banmo.git --recursive
os.chdir('banmo')

# Install required packages
%env FORCE_CUDA=1
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip3 install mediapy
!pip3 install trimesh
!pip3 install pypng
!pip3 install av
!pip3 install pymcubes
!pip3 install kmeans_pytorch
!pip3 install pyyaml pycocotools
!pip3 install fvcore
!pip install pyrender
!pip install -e third_party/softras
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip3 install 'git+https://github.com/facebookresearch/pytorch3d.git'

Cloning into 'banmo'...
remote: Enumerating objects: 1419, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 1419 (delta 51), reused 28 (delta 10), pack-reused 1317
Receiving objects: 100% (1419/1419), 2.38 MiB | 10.84 MiB/s, done.
Resolving deltas: 100% (425/425), done.
Submodule 'third_party/detectron2' (https://github.com/facebookresearch/detectron2) registered for path 'third_party/detectron2'
Submodule 'third_party/kmeans_pytorch' (https://github.com/subhadarship/kmeans_pytorch) registered for path 'third_party/kmeans_pytorch'
Submodule 'third_party/pytorch3d' (https://github.com/facebookresearch/pytorch3d) registered for path 'third_party/pytorch3d'
Cloning into '/content/banmo/third_party/detectron2'...
remote: Enumerating objects: 15270, done.        
remote: Counting objects: 100% (291/291), done.        
remote: Compressing objects: 100% (186/186), done.        
remote: Total 15270 (delta 151), reused 177 (del

# **1- Download data and pre-optimized models**


## 1-1- Download pre-processed data

It downloads the 2D observations (rgb, segmentation mask, optical flow, and densepose images)

In [ ]:
!bash misc/processed/download.sh cat-pikachiu-imgs
#!bash misc/processed/download.sh cat-pikachiu
#!bash misc/processed/download.sh human-cap

--2023-10-23 17:16:00--  https://www.dropbox.com/s/9x9wwop3kcxxe5r/cat-pikachiu-imgs.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/9x9wwop3kcxxe5r/cat-pikachiu-imgs.zip [following]
--2023-10-23 17:16:00--  https://www.dropbox.com/s/raw/9x9wwop3kcxxe5r/cat-pikachiu-imgs.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8b40322977b03662a001d8c3ed.dl.dropboxusercontent.com/cd/0/inline/CGJmkqNijF0IkzxhoKMSQnkuNOEduy3qKsu6s5U4QjN-XHPblsmBOTEqwxvU8vz2CYE1fdGWUGojgujE8bElq9pYzYTgAn-GjK_n989lkIxUPQdD1F_Gf_5QO_26Y12X6XsnJV7U0YswMi-Aju5JrTZ-/file# [following]
--2023-10-23 17:16:01--  https://uc8b40322977b03662a001d8c3ed.dl.dropboxusercontent.com/cd/0/inline/CGJmkqNijF0IkzxhoKMSQnkuNOEduy3qKsu6s5U4QjN-XHPblsmBOTEqwxvU8vz2CYE1fdGWUGo

## 1-2- Download raw videos

It downloads raw videos to the "./raw/" folder

In [ ]:
!bash misc/vid/download.sh cat-pikachiu
!bash misc/vid/download.sh human-cap
!bash misc/vid/download.sh dog-tetres
!bash misc/vid/download.sh cat-coco

## 1-3- Download pre-optimized model

Here, as an exmple, I download the pre-optimized models for a cat. It stores the models in the "./tmp" folder.

In [ ]:
!mkdir -p tmp
os.chdir('tmp')
!wget https://www.dropbox.com/s/qzwuqxp0mzdot6c/cat-pikachiu.npy
!wget https://www.dropbox.com/s/dnob0r8zzjbn28a/cat-pikachiu.pth
!wget https://www.dropbox.com/s/p74aaeusprbve1z/opts.log # flags used at opt time
os.chdir('../')

--2023-10-23 14:34:06--  https://www.dropbox.com/s/qzwuqxp0mzdot6c/cat-pikachiu.npy
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/qzwuqxp0mzdot6c/cat-pikachiu.npy [following]
--2023-10-23 14:34:06--  https://www.dropbox.com/s/raw/qzwuqxp0mzdot6c/cat-pikachiu.npy
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5413b4ba3aec4bdecc6f2a4491.dl.dropboxusercontent.com/cd/0/inline/CGIsMAJ-cWEpazj909RNcXF5pgMS0i6_oBj-yNcjHjA-q88gkW6G6ALORuVwvILjIQHhedB6wdbvbvNO4sByO5AxP6RRFu3dnJMAoee74L0vq0-tv_7VXDsSZqdgq0u50IUKZvEgIkNOjPDDMSOEN8Uu/file# [following]
--2023-10-23 14:34:07--  https://uc5413b4ba3aec4bdecc6f2a4491.dl.dropboxusercontent.com/cd/0/inline/CGIsMAJ-cWEpazj909RNcXF5pgMS0i6_oBj-yNcjHjA-q88gkW6G6ALORuVwvILjIQHhedB6wdbvbvNO4sByO5AxP6

# **2- Experiments**

## 2-1-Trying pre-optimized models

### 2-1-1- Novel view synthesis:
Using their bash scripts, I try to run their "novel view synthesis" task (results will be saved at "tmp/*all.mp4").

In [ ]:
import os
os.chdir('/content/banmo')

# Render novel views
seqname='cat-pikachiu'
!bash scripts/render_nvs.sh 0 $seqname tmp/cat-pikachiu.pth 5 0
# argv[1]: gpu id
# argv[2]: sequence name
# argv[3]: path to the weights
# argv[4]: video id used for pose traj
# argv[5]: video id used for root traj

2023-10-23 15:00:12.354723: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-23 15:00:13.409967: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-23 15:00:14.734002: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-23 15:00:14.740470: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

### 2-1-2- Articulated mesh extraction:
Using their bash scripts, I try to run their "mesh extraction" task (results will be saved at "tmp/*all.mp4").

In [ ]:
import os
os.chdir('/content/banmo')


# Extract articulated meshes and render
seqname='cat-pikachiu'
!bash scripts/render_mgpu.sh 0 $seqname tmp/cat-pikachiu.pth "0 5" 64
# argv[1]: gpu id
# argv[2]: sequence name
# argv[3]: weights path
# argv[4]: video id separated by space
# argv[5]: resolution of running marching cubes (use 256 to get higher-res mesh)

### 2-1-3- Motion retargeting

This is an example of retargeting a source (driven) dog model to a target (driver) cat video.

First, download the pre-trained dog model.

In [ ]:
import os
os.chdir('/content/banmo/tmp')

!wget https://www.dropbox.com/s/wqm7ulxx9qpu85g/shiba-haru-1.npy
!wget https://www.dropbox.com/s/e21vycgoe5y7pio/shiba-haru-1.pth

--2023-10-23 15:40:27--  https://www.dropbox.com/s/wqm7ulxx9qpu85g/shiba-haru-1.npy
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/wqm7ulxx9qpu85g/shiba-haru-1.npy [following]
--2023-10-23 15:40:27--  https://www.dropbox.com/s/raw/wqm7ulxx9qpu85g/shiba-haru-1.npy
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1ea97f97ee7837539c0e0d6345.dl.dropboxusercontent.com/cd/0/inline/CGLx65A0B5WLdrBEzQC1itFvO3x_nacjNH_gdYrkavSxXs-wEQJXEvkBrT4WoVeabmHK04Hufp0I2NMwep25j7Stu4Rg4GoWETvb8K_R-Px0TnFW87I3p7f04r_9Fi2MFDjtOiOnLyHgPIjAo2Ke3QMl/file# [following]
--2023-10-23 15:40:27--  https://uc1ea97f97ee7837539c0e0d6345.dl.dropboxusercontent.com/cd/0/inline/CGLx65A0B5WLdrBEzQC1itFvO3x_nacjNH_gdYrkavSxXs-wEQJXEvkBrT4WoVeabmHK04Hufp0I2NMwep25j7Stu4

Then run retargeting (optimization) on a cat video. Results will be saved at logdir/driver-$seqname-e120-b256/nvs*.mp4.

In [ ]:
import os
os.chdir('/content/banmo')

seqname='cat-pikachiu-sub'
# To speed up data loading, we store images as lines of pixels.
# only needs to run it once per sequence and data are stored
!python preprocess/img2lines.py --seqname $seqname

# Optimization
!bash scripts/template-retarget.sh 0,1 $seqname 10001 "no" "no" tmp/shiba-haru-1.pth
# argv[1]: gpu ids separated by comma
# args[2]: sequence name
# args[3]: port for distributed training
# args[4]: use_human, pass "" for human cse, "no" for quadreped cse
# args[5]: use_symm, pass "" to force x-symmetric shape
# args[6]: driven model

# Extract articulated meshes and render
!bash scripts/render_nvs.sh 0 $seqname logdir/driver-$seqname-e120-b256/params_latest.pth 0 0
# argv[1]: gpu id
# argv[2]: sequence name
# argv[3]: weights path
# argv[4]: video id used for pose traj
# argv[5]: video id used for root traj

2023-10-23 15:41:00.403146: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-23 15:41:01.969527: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-23 15:41:04.103422: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-23 15:41:04.112347: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

### 2-1-4- Adaptation to a new video

Instead of retargeting to a new video, we can adapt a pre-optimized model to a new video, which usually converges faster and more stable than starting from scratch. We show an example of adapting cat-coco to a single video of cat-socks.

First download the pre-trained cat-coco model.

In [ ]:
import os
os.chdir('/content/banmo/tmp')

!wget https://www.dropbox.com/s/tvc9rpef9fgb7vp/cat-coco.npy
!wget https://www.dropbox.com/s/iwosbezgfwcr54w/cat-coco.pth

--2023-10-23 15:48:33--  https://www.dropbox.com/s/tvc9rpef9fgb7vp/cat-coco.npy
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/tvc9rpef9fgb7vp/cat-coco.npy [following]
--2023-10-23 15:48:34--  https://www.dropbox.com/s/raw/tvc9rpef9fgb7vp/cat-coco.npy
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca57c07e2bbd4f3a65377fb0916.dl.dropboxusercontent.com/cd/0/inline/CGIi2_F6T3MJiYYrxxymh4qwuaqB2mHsrcG3drq9xZFhvT4TVipXpAa7t7wfbGpOTb1ptc-1xUq8i2Aqkc94txsfoI5wK0gSlzHhGutL4muYxnhkW9lr1oHjaNjuf0taU7Hz2IGdAVVuAP3EapZGJ4hR/file# [following]
--2023-10-23 15:48:34--  https://uca57c07e2bbd4f3a65377fb0916.dl.dropboxusercontent.com/cd/0/inline/CGIi2_F6T3MJiYYrxxymh4qwuaqB2mHsrcG3drq9xZFhvT4TVipXpAa7t7wfbGpOTb1ptc-1xUq8i2Aqkc94txsfoI5wK0gSlzHhGu

Then download and run optimization on a video of cat-socks

In [ ]:
import os
os.chdir('/content/banmo')

seqname='cat-socks'
!bash misc/processed/download.sh $seqname
!python preprocess/img2lines.py --seqname $seqname
!bash scripts/template-prior-model.sh 0,1 $seqname 10001 "no" "no" tmp/cat-coco.pth
!bash scripts/render_mgpu.sh 0 $seqname logdir/prior-$seqname-e30-b256-ft2/params_latest.pth "0" 256

--2023-10-23 15:49:50--  https://www.dropbox.com/s/j9425to9sdx6l08/cat-socks006.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/j9425to9sdx6l08/cat-socks006.zip [following]
--2023-10-23 15:49:51--  https://www.dropbox.com/s/raw/j9425to9sdx6l08/cat-socks006.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3a7e502ed291fe29156112c57b.dl.dropboxusercontent.com/cd/0/inline/CGJ36IFSGFKsdf_uN2pn3HKLrG2APhv98iAxBGTCcDQzKIp97T9Zqaznwi7z9s3mChILzIBBW0Zpd1nH9E8zC3P_M25AI7TDX-z1Kfz8-zXGSbypm-aFxW3bpRQtaByWVHc7nGj_Rqn_PF6lTEfI6Dp0/file# [following]
--2023-10-23 15:49:52--  https://uc3a7e502ed291fe29156112c57b.dl.dropboxusercontent.com/cd/0/inline/CGJ36IFSGFKsdf_uN2pn3HKLrG2APhv98iAxBGTCcDQzKIp97T9Zqaznwi7z9s3mChILzIBBW0Zpd1nH9E8zC3P_M2

### 2-1-5- Synthetic data

Download animated mesh sequences

In [ ]:
import os
os.chdir('/content/banmo/database/eagle')

!wget https://www.dropbox.com/sh/xz8kckfq817ggqd/AADIhtb1syWhDQeY8xa9Brc0a -O eagle.zip
!unzip eagle.zip;

import os
os.chdir('/content/banmo/database/hands')

!wget https://www.dropbox.com/sh/kbobvwtt51bl165/AAC4d-tbJ5PR6XQIUjbk3Qe2a -O hands.zip
!unzip hands.zip;

--2023-10-23 16:14:44--  https://www.dropbox.com/sh/xz8kckfq817ggqd/AADIhtb1syWhDQeY8xa9Brc0a
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /sh/raw/xz8kckfq817ggqd/AADIhtb1syWhDQeY8xa9Brc0a [following]
--2023-10-23 16:14:45--  https://www.dropbox.com/sh/raw/xz8kckfq817ggqd/AADIhtb1syWhDQeY8xa9Brc0a
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd188b474f4e0f5bc63ce282a1f.dl.dropboxusercontent.com/zip_download_get/Bpjs1BtHK6y1v7_wRrISUCaAAvZ_EbnWhbL5pBnU3hvOhvJ8u0kJxn9hTgmQeT6_7XlmAeBrkCRjcKprf_dSQv51phpLvTZXXeReiW7PCPAGmA# [following]
--2023-10-23 16:14:46--  https://ucd188b474f4e0f5bc63ce282a1f.dl.dropboxusercontent.com/zip_download_get/Bpjs1BtHK6y1v7_wRrISUCaAAvZ_EbnWhbL5pBnU3hvOhvJ8u0kJxn9hTgmQeT6_7XlmAeBrkCRjcKprf_dSQv51phpLvTZ

Render image data and prepare mesh ground-truth

In [ ]:
import os
os.chdir('/content/banmo')

!bash scripts/synthetic/render_eagle.sh
!bash scripts/synthetic/render_hands.sh

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
0
1
2
3
4
5
6
7
8
9


To run optimization on eagle:

In [ ]:
import os
os.chdir('/content/banmo')

seqname='a-eagle'
# store as lines
!python preprocess/img2lines.py --seqname $seqname
# optimization
!bash scripts/template-known-cam.sh 0,1 $seqname 10001 "no" "no"
# extract articulated meshes
!bash scripts/render_mgpu.sh 0 $seqname logdir/known-cam-$seqname-e120-b256-ft2/params_latest.pth "0" 256

## 2-2- Novel view rendering

This is an example for novel view rendering given a model optimized from 11 casual cat videos

In [ ]:
import os
os.chdir('/content/banmo')


import sys

if sys.version_info[0] == 2:
    import _pickle as cPickle
else:
    import pickle

# import libs
import numpy as np
import cv2
import torch
from absl import flags
from collections import defaultdict
import mediapy
from google.colab.patches import cv2_imshow

from nnutils import banmo
from utils.io import extract_data_info
from dataloader import frameloader
from nnutils.geom_utils import get_near_far, sample_xy, K2inv, raycast, chunk_rays
from nnutils.train_utils import v2s_trainer
from nnutils.rendering import render_rays
opts = flags.FLAGS

def construct_rays_nvs(target_size, rtks, near_far, rndmask, device):
    """
    rndmask: controls which pixel to render
    """
    bs = rtks.shape[0]
    rtks = torch.Tensor(rtks).to(device)
    rndmask = torch.Tensor(rndmask).to(device).view(-1)>0

    img_size = max(target_size)
    _, xys = sample_xy(img_size, bs, 0, device, return_all=True)
    xys=xys.view(img_size,img_size,2)[:target_size[0], :target_size[1]].view(1,-1,2)
    xys = xys[:,rndmask]
    Rmat = rtks[:,:3,:3]
    Tmat = rtks[:,:3,3]
    Kinv = K2inv(rtks[:,3])
    rays = raycast(xys, Rmat, Tmat, Kinv, near_far)
    return rays

# load flags
args = opts.read_flags_from_files(['--flagfile=tmp/opts.log',
                                   '--model_path=tmp/cat-pikachiu.pth',
                                   '--seqname=cat-pikachiu',
                                   '--nouse_corresp',
                                   '--nouse_unc',
                                   '--perturb=0',
                                   '--chunk=2048'])
unknown_flags, unparsed_args = opts._parse_args(args, known_only=True)
opts.mark_as_parsed()
opts.validate_all_flags()

# load model
trainer = v2s_trainer(opts, is_eval=True)
data_info = trainer.init_dataset()
trainer.define_model(data_info)

model = trainer.model
model.eval()

nerf_models = model.nerf_models
embeddings = model.embeddings # positional encodings

# workers: 8
# pairs: 256
init:0, end:-1
9272 pairs of images
init:0, end:-1
9266 pairs of images
init:0, end:-1
9072 pairs of images
init:0, end:-1
9282 pairs of images
init:0, end:-1
9120 pairs of images
init:0, end:-1
9184 pairs of images
init:0, end:-1
9118 pairs of images
init:0, end:-1
9230 pairs of images
init:0, end:-1
9272 pairs of images
init:0, end:-1
9280 pairs of images
init:0, end:-1
9256 pairs of images
# workers: 8
# pairs: 256
init:0, end:-1
9272 pairs of images
init:0, end:-1
9266 pairs of images
init:0, end:-1
9072 pairs of images
init:0, end:-1
9282 pairs of images
init:0, end:-1
9120 pairs of images
init:0, end:-1
9184 pairs of images
init:0, end:-1
9118 pairs of images
init:0, end:-1
9230 pairs of images
init:0, end:-1
9272 pairs of images
init:0, end:-1
9280 pairs of images
init:0, end:-1
9256 pairs of images
init:0, end:-1
121 pairs of images
init:0, end:-1
112 pairs of images
init:0, end:-1
125 pairs of images
init:0, end:-1
38 pairs of images
init:0, end:-1
94

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings

AttributeError: ignored

In [ ]:
#@title Rendering parameters { run: "auto", vertical-output: true }
vidid = 0 #@param {type:"slider", min:0, max:10, step:1}
image_scale = 0.025 #@param {type:"slider", min:0.025, max:0.5, step:0.005}
use_turntable = "False" #@param ["True", "False"]
bullet_time = 0
use_bullettime = "False" #@param ["True", "False"]
bullet_time = 0 #@param {type:"slider", min:0, max:1, step:0.01}


# select video to render
vidid = vidid # 0 to 10

# select rendering resolution
raw_res = (1080,1920) # h, w
target_size = (int(raw_res[0]*image_scale),
               int(raw_res[1]*image_scale))

# select rendering mode
if use_turntable =='True':
  use_turntable = True
else:
  use_turntable = False
if use_bullettime == 'True':
  bullet_time = bullet_time # 0 to 1, if < 0 render without bulet time effect
else:
  bullet_time = -1

# get frame id
start_idx = model.data_offset[vidid]
end_idx = model.data_offset[vidid+1]
sample_idx = np.asarray(range(start_idx,end_idx))
frameid = torch.Tensor(sample_idx).to(model.device).long()
bs = len(frameid)

# query root poses
# bs, 4,4 (R|T) # extrinsics
#         (f|p) # intrinsics
with torch.no_grad():
  rtks = torch.eye(4)[None].repeat(bs,1,1)
  root_rts = model.nerf_root_rts(frameid)
  rtk_base = model.create_base_se3(bs, model.device)
  rtks[:,:3] = model.refine_rt(rtk_base, root_rts)
  rtks[:,3,:] = model.ks_param[vidid]
  rtks[:,3] = rtks[:,3]*image_scale

# optional: turn table
if use_turntable:
  for i in range(bs):
    rot_turntable = cv2.Rodrigues(np.asarray([0.,i*2*np.pi/bs,0.]))[0]
    rot_turntable = torch.Tensor(rot_turntable)
    rtks[i,:3,:3] = rot_turntable @ rtks[0,:3,:3]
    rtks[i,:3,3] = rtks[0,:3,3]

# compute near-far plane
near_far = torch.zeros(bs,2).to(model.device)
vars_np = {}
vars_np['rtk'] = rtks.cpu().numpy()
vars_np['idk'] = np.ones(bs)
near_far = get_near_far(near_far,
                        vars_np,
                        pts=model.latest_vars['mesh_rest'].vertices)

# render
frames = []
for i in range(len(frameid)):
  render_frame_id = frameid[i:i+1]
  if bullet_time >=0:
    render_frame_id = frameid[int(bullet_time * (bs-1))][None]
  print('rendering video %02d, frame %03d/%03d'%(vidid, i,len(frameid)))

  rndmask = np.ones((target_size[0], target_size[1]))
  rays = construct_rays_nvs(target_size, rtks[i:i+1],
                                  near_far[i:i+1], rndmask, model.device)
  # query env code
  rays['env_code'] = model.env_code(render_frame_id)[:,None]
  rays['env_code'] = rays['env_code'].repeat(1,rays['nsample'],1)

  # query deformation
  time_embedded = model.pose_code(render_frame_id)[:,None]
  rays['time_embedded'] = time_embedded.repeat(1,rays['nsample'],1)
  bone_rts = model.nerf_body_rts(render_frame_id)
  rays['bone_rts'] = bone_rts.repeat(1,rays['nsample'],1)
  model.update_delta_rts(rays)

  with torch.no_grad():
  # render images only
    results=defaultdict(list)
    bs_rays = rays['bs'] * rays['nsample'] #
    for j in range(0, bs_rays, opts.chunk):
      rays_chunk = chunk_rays(rays,j,opts.chunk)
      rendered_chunks = render_rays(nerf_models,
                  embeddings,
                  rays_chunk,
                  N_samples = opts.ndepth,
                  perturb=0,
                  noise_std=0,
                  chunk=opts.chunk, # chunk size is effective in val mode
                  use_fine=True,
                  img_size=model.img_size,
                  obj_bound = model.latest_vars['obj_bound'],
                  render_vis=True,
                  opts=opts,
                  )
      for k, v in rendered_chunks.items():
        results[k] += [v]

    for k, v in results.items():
      v = torch.cat(v, 0)
      v = v.view(rays['nsample'], -1)
      results[k] = v

    rgb = results['img_coarse'].cpu().numpy().reshape(target_size[0], target_size[1],3)
    sil = results['sil_coarse'][...,0].cpu().numpy().reshape(target_size[0], target_size[1])
    rgb[sil<0.5] = 0
    frames.append(rgb)

mediapy.show_video(frames, fps=10)
cv2_imshow(sil*255)
cv2_imshow(rgb[:,:,::-1]*255)